<a href="https://colab.research.google.com/github/hemanthkumarkatta/Real-Time-Stock-Market-Analysis-Past-Vs-Future-Data/blob/main/Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from datetime import datetime
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
# Function to fetch stock prices (open, close, high, low)
def fetch_stock_data(symbol, api_key, years=10):
    stock_data = []
    current_year = datetime.now().year
    for year in range(current_year, current_year - years, -1):
        url = f'https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?from={year}-01-01&to={year}-12-31&apikey={api_key}'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if 'historical' in data:
                yearly_data = [{
                    'Date': item['date'],
                    'Open': item['open'],
                    'Close': item['close'],
                    'High': item['high'],
                    'Low': item['low']
                } for item in data['historical']]
                stock_data.extend(yearly_data)
            else:
                print(f"No historical data found for {symbol} in {year}")
        else:
            print(f"Error: {response.status_code} for year {year}")
    return pd.DataFrame(stock_data)
# Function to fetch dividends data using the correct endpoint
# API Key and Stock Symbol
api_key = 'BNAH5V0CfJxuxGPA4ePVx0eo0Kal32p8'
symbol = 'JPM'
# Fetch stock prices and dividends data
stock_data = fetch_stock_data(symbol, api_key, years=5)
print(stock_data)

            Date    Open   Close    High     Low
0     2025-02-10  276.15  271.04  276.18  269.59
1     2025-02-07  277.27  275.80  278.55  274.71
2     2025-02-06  272.45  276.90  276.98  271.19
3     2025-02-05  268.91  270.43  270.64  268.33
4     2025-02-04  269.84  267.94  269.84  266.83
...          ...     ...     ...     ...     ...
1026  2021-01-08  135.97  136.02  136.35  134.12
1027  2021-01-07  135.69  135.87  138.19  134.92
1028  2021-01-06  129.88  131.55  132.77  127.88
1029  2021-01-05  124.99  125.65  126.30  123.77
1030  2021-01-04  127.50  125.87  127.86  124.78

[1031 rows x 5 columns]


In [ ]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031 entries, 0 to 1030
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1031 non-null   object 
 1   Open    1031 non-null   float64
 2   Close   1031 non-null   float64
 3   High    1031 non-null   float64
 4   Low     1031 non-null   float64
dtypes: float64(4), object(1)
memory usage: 40.4+ KB


In [ ]:
stock_data.describe()

,Open,Close,High,Low
count,1031.000000,1031.000000,1031.000000,1031.000000
mean,160.766654,160.841319,162.276683,159.299602
std,35.564123,35.612694,35.795698,35.389744
min,101.800000,101.960000,104.540000,101.280000
25%,136.500000,136.665000,138.175000,135.200000
50%,152.870000,152.910000,154.140000,151.360000
75%,173.470000,173.995000,175.095000,172.200000
max,277.270000,276.900000,278.550000,274.710000


In [ ]:
'''def preprocess_data(data):
    data = data.copy()

    # Handle date parsing with a flexible format
    data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

    # Drop rows where the date could not be parsed
    data = data.dropna(subset=['Date'])

    # Fill missing values in other columns
    data = data.ffill()

    # Ensure there are enough valid rows
    if len(data) < 2:
        raise ValueError("Dataframe has less than 2 non-NaN rows after preprocessing.")

    return data
dataset = preprocess_data(stock_data)'''

'def preprocess_data(data):\n    data = data.copy()\n\n    # Handle date parsing with a flexible format\n    data[\'Date\'] = pd.to_datetime(data[\'Date\'], errors=\'coerce\')\n\n    # Drop rows where the date could not be parsed\n    data = data.dropna(subset=[\'Date\'])\n\n    # Fill missing values in other columns\n    data = data.ffill()\n\n    # Ensure there are enough valid rows\n    if len(data) < 2:\n        raise ValueError("Dataframe has less than 2 non-NaN rows after preprocessing.")\n\n    return data\ndataset = preprocess_data(stock_data)'

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.make_holidays import make_holidays_df
from sklearn.metrics import mean_absolute_error, mean_squared_error
from prophet.diagnostics import cross_validation, performance_metrics

def forecast_from_dec_1st(data, column_name):

    # Ensure the 'Date' column is in datetime format
    data['Date'] = pd.to_datetime(data['Date'])

    # Filter data starting from the past year
    one_year_ago = pd.Timestamp.now() - pd.DateOffset(years=1)
    data_last_year = data[data['Date'] >= one_year_ago]

    # Prepare the data for Prophet
    df = data_last_year[['Date', column_name]].rename(columns={'Date': 'ds', column_name: 'y'})

    # Handle outliers in the data by capping extreme values
    df['y'] = df['y'].clip(lower=df['y'].quantile(0.01), upper=df['y'].quantile(0.99))

    # Initialize the Prophet model with additional options
    holidays = make_holidays_df(year_list=list(range(2015, 2025)), country='US')
    model = Prophet(holidays=holidays, seasonality_mode='multiplicative',
                    yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True)

    # Add custom monthly seasonality
    model.add_seasonality(name='monthly', period=30.5, fourier_order=20)

    # Train the model
    model.fit(df)

    # Cross-validation and performance metrics
    cv_results = cross_validation(model, initial='180 days', period='2 days', horizon='90 days')
    performance = performance_metrics(cv_results)
    print(f"Cross-validation metrics for {column_name}:")
    print(performance[['horizon', 'rmse', 'mape']])

    # Create a future dataframe for predictions starting from December 1st
    future = model.make_future_dataframe(periods=365)
    future = future[future['ds'] >= pd.Timestamp('2024-12-01')]

    # Remove weekends from the future dataframe
    future = future[future['ds'].dt.weekday < 5]  # 0=Monday, 4=Friday

    # Predict future values
    forecast = model.predict(future)

    # Apply post-processing to avoid extreme predictions
    forecast['yhat'] = forecast['yhat'].clip(lower=forecast['yhat'].quantile(0.01), upper=forecast['yhat'].quantile(0.99))

    # Extract and rename relevant columns
    forecast = forecast[['ds', 'yhat']].rename(columns={'yhat': f'Predicted_{column_name}'})
    return forecast

# Example function to forecast all columns starting from December 1st
def forecast_all_columns_from_dec_1st(data):
    columns_to_forecast = ['Open', 'Close', 'Low', 'High']
    forecasts = []
    for column in columns_to_forecast:
        forecast = forecast_from_dec_1st(data, column)
        forecasts.append(forecast)
    # Combine all forecasts
    result = forecasts[0]
    for forecast in forecasts[1:]:
        result = pd.merge(result, forecast, on='ds')
    # Rename 'ds' to 'Date' for consistency
    result = result.rename(columns={'ds': 'Date'})
    return result
# Forecast stock data starting from December 1st
updated_forecast = forecast_all_columns_from_dec_1st(stock_data)

# Display the resulting forecasted DataFrame
print(updated_forecast)

# Output the combined dataset
stocks = pd.DataFrame(updated_forecast)
stocks

DEBUG:cmdstanpy:input tempfile: /tmp/tmp0en5lt2w/d319bo9k.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0en5lt2w/rs5crgng.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=91815', 'data', 'file=/tmp/tmp0en5lt2w/d319bo9k.json', 'init=/tmp/tmp0en5lt2w/rs5crgng.json', 'output', 'file=/tmp/tmp0en5lt2w/prophet_modelh6yr9wmr/prophet_model-20250211093416.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:34:16 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:34:16 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Making 48 forecasts with cutoffs between 2024-08-10 00:00:00 and 2024-11-12 00:00:00


  0%|          | 0/48 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmp0en5lt2w/ovc1vuvn.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0en5lt2w/bsnszstz.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=92043', 'data', 'file=/tmp/tmp0en5lt2w/ovc1vuvn.json', 'init=/tmp/tmp0en5lt2w/bsnszstz.json', 'output', 'file=/tmp/tmp0en5lt2w/prophet_model8p1cpbic/prophet_model-20250211093416.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:34:16 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:34:16 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0en5lt2w/tkurqa5d.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0en5lt2w/c5uwjsc_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

Cross-validation metrics for Open:
   horizon       rmse      mape
0   9 days  32.730614  0.097300
1  10 days  39.939684  0.118702
2  11 days  47.359215  0.139587
3  12 days  55.884966  0.163833
4  13 days  61.763960  0.184832
..     ...        ...       ...
77 86 days  80.847546  0.171653
78 87 days  82.346586  0.170900
79 88 days  83.106505  0.169770
80 89 days  83.506632  0.168975
81 90 days  82.513200  0.165982

[82 rows x 3 columns]


DEBUG:cmdstanpy:input tempfile: /tmp/tmp0en5lt2w/0bi9kk8s.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0en5lt2w/j47lghh3.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=78340', 'data', 'file=/tmp/tmp0en5lt2w/0bi9kk8s.json', 'init=/tmp/tmp0en5lt2w/j47lghh3.json', 'output', 'file=/tmp/tmp0en5lt2w/prophet_modeltjyq_vap/prophet_model-20250211093429.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:34:29 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:34:29 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Making 48 forecasts with cutoffs between 2024-08-10 00:00:00 and 2024-11-12 00:00:00


...

DEBUG:cmdstanpy:input tempfile: /tmp/tmp0en5lt2w/mb_al7ll.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0en5lt2w/1n7ayedv.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=47200', 'data', 'file=/tmp/tmp0en5lt2w/mb_al7ll.json', 'init=/tmp/tmp0en5lt2w/1n7ayedv.json', 'output', 'file=/tmp/tmp0en5lt2w/prophet_modeld0raprzi/prophet_model-20250211093455.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:34:55 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:34:55 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Making 48 forecasts with cutoffs between 2024-08-10 00:00:00 and 2024-11-12 00:00:00


  0%|          | 0/48 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmp0en5lt2w/3qxw6hsh.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0en5lt2w/txa9pwd4.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=96519', 'data', 'file=/tmp/tmp0en5lt2w/3qxw6hsh.json', 'init=/tmp/tmp0en5lt2w/txa9pwd4.json', 'output', 'file=/tmp/tmp0en5lt2w/prophet_modell9p3ki5n/prophet_model-20250211093456.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:34:56 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:34:56 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0en5lt2w/ijbqk4b_.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0en5lt2w/d5wrfj94.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

Cross-validation metrics for High:
   horizon       rmse      mape
0   9 days  30.764996  0.087136
1  10 days  36.794241  0.106079
2  11 days  42.984475  0.125006
3  12 days  51.399192  0.147878
4  13 days  57.394124  0.166103
..     ...        ...       ...
77 86 days  79.565166  0.164376
78 87 days  80.076595  0.161672
79 88 days  80.227265  0.159723
80 89 days  82.616802  0.160660
81 90 days  83.409887  0.157413

[82 rows x 3 columns]
          Date  Predicted_Open  Predicted_Close  Predicted_Low  Predicted_High
0   2024-12-02      247.321245       245.256999     244.223423      248.581421
1   2024-12-03      250.009453       247.381278     247.137562      250.347886
2   2024-12-04      246.284496       245.142360     243.540163      248.327421
3   2024-12-05      245.018161       243.302946     242.864893      245.250234
4   2024-12-06      244.149217       245.391318     242.744913      247.145315
..         ...             ...              ...            ...             ...
303 2

,Date,Predicted_Open,Predicted_Close,Predicted_Low,Predicted_High
0,2024-12-02,247.321245,245.256999,244.223423,248.581421
1,2024-12-03,250.009453,247.381278,247.137562,250.347886
2,2024-12-04,246.284496,245.142360,243.540163,248.327421
3,2024-12-05,245.018161,243.302946,242.864893,245.250234
4,2024-12-06,244.149217,245.391318,242.744913,247.145315
...,...,...,...,...,...
303,2026-02-04,364.592419,362.689774,362.069461,366.317067
304,2026-02-05,364.203277,361.374197,362.724452,363.128254
305,2026-02-06,364.431150,364.618475,364.246248,367.108238
306,2026-02-09,364.592419,364.618475,364.325462,367.108238


In [ ]:
from datetime import date, timedelta
import pandas as pd

# Get yesterday's date
yesterday = date.today() - timedelta(days=1)

# Filter the 'result' DataFrame for yesterday's date
yesterday_data = updated_forecast[updated_forecast['Date'] == pd.Timestamp(yesterday)]
yesterday_data

,Date,Predicted_Open,Predicted_Close,Predicted_Low,Predicted_High
46,2025-02-10,271.555812,271.343345,269.248553,273.367695


In [ ]:
january_2025_data = updated_forecast[(pd.to_datetime(updated_forecast['Date']).dt.year == 2025) & (pd.to_datetime(updated_forecast['Date']).dt.month == 1)]
print(january_2025_data)

         Date  Predicted_Open  Predicted_Close  Predicted_Low  Predicted_High
21 2025-01-02      240.598472       237.900616     236.997328      241.143950
22 2025-01-03      239.713288       241.379333     237.901725      242.555924
23 2025-01-06      240.500377       242.960831     238.839362      244.993345
24 2025-01-07      242.795944       242.709565     240.382679      245.338942
25 2025-01-08      243.447029       243.832367     240.809677      245.548485
26 2025-01-10      246.450205       244.247123     240.931959      248.602932
27 2025-01-13      244.733359       246.515183     242.976610      247.957354
28 2025-01-14      247.803945       250.712397     247.224311      251.662323
29 2025-01-15      251.630554       253.798185     248.693777      255.197689
30 2025-01-16      253.396277       254.857355     252.351399      255.374889
31 2025-01-17      253.242093       254.971273     250.485562      256.919152
32 2025-01-21      258.254370       259.389213     256.621530   

In [ ]:
from datetime import date, timedelta
import pandas as pd

# ... (Your existing code for fetch_stock_data, preprocess_data,
# forecast_prophet_last_year, etc.) ...

# Get today's date
today = date.today()

# Calculate the start date for the last 10 days
start_date = today - timedelta(days=20)

# Filter the 'result' DataFrame for the last 10 days
last_10_days_predictions = updated_forecast[(updated_forecast['Date'] >= pd.Timestamp(start_date)) & (updated_forecast['Date'] <= pd.Timestamp(today))]

# Display the predictions for the last 10 days
print(last_10_days_predictions)

         Date  Predicted_Open  Predicted_Close  Predicted_Low  Predicted_High
33 2025-01-22      258.738906       261.124722     256.786082      261.500104
34 2025-01-23      262.356411       265.094637     261.832224      265.523181
35 2025-01-24      263.355971       264.399507     261.747147      266.256332
36 2025-01-27      264.206398       265.686760     262.740515      266.835209
37 2025-01-28      265.524228       266.741821     264.114219      267.596585
38 2025-01-29      267.243943       268.490579     266.292688      270.333979
39 2025-01-30      268.360746       269.675001     266.763158      271.441583
40 2025-01-31      269.136013       267.268817     266.029042      270.977380
41 2025-02-03      266.908132       267.109555     264.638268      269.672214
42 2025-02-04      266.855161       266.788508     265.456905      267.897111
43 2025-02-05      267.874944       269.928814     266.782406      271.030623
44 2025-02-06      272.074437       270.046936     269.407748   

In [ ]:
from datetime import date, timedelta
import pandas as pd

# Get today's date
today = date.today()

# Filter the 'result' DataFrame for today's date
today_data = updated_forecast[updated_forecast['Date'] == pd.Timestamp(today)]

# If today's data is not found, display a message
if today_data.empty:
    print(f"No prediction found for today's date: {today}")
else:
    print(f"Today's Prediction:\n{today_data}")

Today's Prediction:
         Date  Predicted_Open  Predicted_Close  Predicted_Low  Predicted_High
47 2025-02-11      271.359929       268.610906     269.609037      271.161812


In [ ]:
from datetime import date, timedelta
import pandas as pd

# Get today's date
today = date.today()

# Calculate the end date for the next 10 days
end_date = today + timedelta(days=10)

# Convert 'Date' column to datetime if it's not already
updated_forecast['Date'] = pd.to_datetime(updated_forecast['Date'])

# Filter the 'result' DataFrame for the next 10 days
next_10_days_predictions = updated_forecast[(updated_forecast['Date'] > pd.Timestamp(today)) & (updated_forecast['Date'] <= pd.Timestamp(end_date))]

# Display the predictions for the next 10 days
print(next_10_days_predictions)

         Date  Predicted_Open  Predicted_Close  Predicted_Low  Predicted_High
48 2025-02-12      273.352218       272.673381     273.148190      273.617667
49 2025-02-13      271.589858       271.906914     271.915192      273.011324
50 2025-02-14      276.060463       274.654108     275.215300      277.193809
51 2025-02-17      277.177839       273.280151     274.824385      276.478241
52 2025-02-18      276.087077       274.718293     276.522734      276.018509
53 2025-02-19      277.741490       276.606405     277.076944      278.881675
54 2025-02-20      280.301751       276.266108     279.509093      277.855383
55 2025-02-21      278.561653       279.003251     279.519756      279.260925


In [ ]:
!pip install gspread
!pip install oauth2client

In [ ]:
!ls -l stellar-arcadia-445904-q3-128839bb6df7.json


-rw-r--r-- 1 root root 2393 Feb 11 09:37 stellar-arcadia-445904-q3-128839bb6df7.json


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
# Define the API scope
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

# Authenticate using the JSON key file
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/stellar-arcadia-445904-q3-128839bb6df7.json', scope)

# Authorize gspread to access Google Sheets
client = gspread.authorize(creds)

# Open the Google Sheet by name
sheet = client.open('stockmarket').sheet1
updated_forecast['Date'] = updated_forecast['Date'].astype(str)
rows = [updated_forecast.columns.tolist()] + updated_forecast.values.tolist()

# Clear existing content (optional)
sheet.clear()

# Write data to the sheet
sheet.update(rows)
print("Data uploaded successfully!")

Data uploaded successfully!


In [ ]:
# Read all rows as a list of dictionaries
data = sheet.get_all_records()
print(data)


[{'Date': '2024-12-02', 'Predicted_Open': 247.3212452, 'Predicted_Close': 245.2569994, 'Predicted_Low': 244.223423, 'Predicted_High': 248.5814206}, {'Date': '2024-12-03', 'Predicted_Open': 250.0094527, 'Predicted_Close': 247.3812779, 'Predicted_Low': 247.1375624, 'Predicted_High': 250.3478861}, {'Date': '2024-12-04', 'Predicted_Open': 246.2844958, 'Predicted_Close': 245.1423602, 'Predicted_Low': 243.5401629, 'Predicted_High': 248.3274213}, {'Date': '2024-12-05', 'Predicted_Open': 245.0181613, 'Predicted_Close': 243.3029455, 'Predicted_Low': 242.8648933, 'Predicted_High': 245.2502339}, {'Date': '2024-12-06', 'Predicted_Open': 244.1492171, 'Predicted_Close': 245.3913181, 'Predicted_Low': 242.7449135, 'Predicted_High': 247.1453149}, {'Date': '2024-12-09', 'Predicted_Open': 244.3146722, 'Predicted_Close': 243.783857, 'Predicted_Low': 242.5991243, 'Predicted_High': 245.9859018}, {'Date': '2024-12-10', 'Predicted_Open': 244.2145238, 'Predicted_Close': 241.5791897, 'Predicted_Low': 241.260448